# Cálculo de ratios

Cálculo de los ratios financieros a partir de la información contable y bursátil disponible, que utilizaremos como variables explicativas del modelo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
! ls -n ../tablas

total 12084
-rw-rw-r-- 1 1000 1000 9992731 jul  9 23:37 banks_cotiz.csv
-rw-rw-r-- 1 1000 1000 1111021 ago  9 00:38 dataformodel.csv
-rw-rw-r-- 1 1000 1000  578247 jul  9 23:37 mergedbalres.csv
-rw-rw-r-- 1 1000 1000  683261 jul 21 00:03 registros_etiq.csv


In [3]:
data = pd.read_csv('../tablas/registros_etiq.csv')

In [4]:
len(data)

1541

In [5]:
list(data.columns)

['Ticker',
 'SimFinId',
 'Currency',
 'Fiscal Year',
 'Fiscal Period',
 'Report Date',
 'Publish Date',
 'Restated Date',
 'Shares (Basic)',
 'Shares (Diluted)',
 'Cash, Cash Equivalents & Short Term Investments',
 'Interbank Assets',
 'Short & Long Term Investments',
 'Accounts & Notes Receivable',
 'Net Loans',
 'Net Fixed Assets',
 'Total Assets',
 'Total Deposits',
 'Short Term Debt',
 'Long Term Debt',
 'Total Liabilities',
 'Preferred Equity',
 'Share Capital & Additional Paid-In Capital',
 'Treasury Stock',
 'Retained Earnings',
 'Total Equity',
 'Total Liabilities & Equity',
 'Revenue',
 'Provision for Loan Losses',
 'Net Revenue after Provisions',
 'Total Non-Interest Expense',
 'Operating Income (Loss)',
 'Non-Operating Income (Loss)',
 'Pretax Income (Loss)',
 'Income Tax (Expense) Benefit, Net',
 'Income (Loss) from Continuing Operations',
 'Net Extraordinary Gains (Losses)',
 'Net Income',
 'Net Income (Common)',
 'Price_d',
 'Publish_Date+90',
 'Price_d+90',
 'DJUSF_d',
 

### Capitalización bursátil

In [6]:
data['Capit']=data['Shares (Basic)']*data['Price_d']

In [7]:
len(data[data['Capit'].isnull()])

58

### PER

In [8]:
data['PER'] = data['Net Income (Common)']/data['Capit']
len(data[data['PER'].isnull()])

58

### Price / Book Value

In [9]:
data['PBC'] = data['Capit']/data['Total Equity']
len(data[data['PBC'].isnull()])

58

### Rentabilidad por dividendo

### ROE

In [10]:
data['ROE'] = data['Net Income (Common)']/data['Total Equity']
len(data[data['ROE'].isnull()])

0

### ROA

In [11]:
data['ROA'] = data['Net Income (Common)']/data['Total Assets']
len(data[data['ROA'].isnull()])

0

### Deuda total / FFPP

In [12]:
data['Deuda_FFPP'] = (data['Short Term Debt'] + data['Long Term Debt'])/data['Total Equity']
len(data[data['Deuda_FFPP'].isnull()])

395

### Deuda corto plazo / FFPP

In [13]:
data['DeudaCP_FFPP'] = data['Short Term Debt'] / data['Total Equity']
len(data[data['DeudaCP_FFPP'].isnull()])

159

### Dotación en el periodo

In [14]:
data['Dot_dudosos'] = data['Provision for Loan Losses']/data['Total Assets']
len(data[data['Dot_dudosos'].isnull()])

51

## Variaciones en los ratios

A continuación vamos a incorporar como variable la variación de cada ratio con respecto al periodo anterior. Para ello primero aseguramos el orden adecuado de la tabla

In [15]:
data.sort_values(['Ticker','Fiscal Year','Fiscal Period'], ascending=True, inplace=True)

Despues incorporamos en cada fila el valor del ratio del periodo anterior, siempre que el Ticker no varíe.

In [16]:
#Definimos una función que genere subtablas por Ticker y en ellas haga un shift para incorporar en cada línea el
#valor de un ratio dado del periodo anterior.

def ratio_ant(ratio,data):
    table_1=pd.DataFrame()
    for ticker in data.Ticker.unique():
        aux_table=data[data.Ticker==ticker]
        aux_table['var_%s'%(ratio)]=(aux_table[ratio].shift(1)/aux_table[ratio])-1
        table_1=table_1.append(aux_table)
    return table_1

In [17]:
#Aplicamos la función anterior para cada ratio

for column in ['PER','PBC','ROE','ROA','Deuda_FFPP','DeudaCP_FFPP','Dot_dudosos']:
    data=ratio_ant(column,data)

/home/pipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Valor relativo de cada ratio

Una posible mejora del modelo consiste en considerar cada uno de los ratios no por su valor absoluto, sino como un valor relativo con respecto al resto de compañias del periodo.
Para ello vamos a obtener para cada periodo el decil que ocupa cada ratio de cada compañía con respecto a las demás.

In [19]:
data = data.astype({'Fiscal Year':str})
data['periodo'] = data['Fiscal Year'] + data['Fiscal Period']

In [18]:
#Definimos una función que genere subtablas por periodo y en ellas calcule el decil de cada ratio de cada compañia

#def decil_ratio(periodo,data):

SyntaxError: unexpected EOF while parsing (<ipython-input-18-9a550d9e3e4b>, line 3)

In [ ]:
data.to_csv('../tablas/dataformodel.csv', index=False)